In [ ]:
import sys
sys.path.append("..")

import skimage.data
import cupy as cp
import numpy as np

from utils import measure_vsnr_cuda, measure_vsnr_numpy, measure_vsnr_cupy, stripes_addition, curtains_addition, print_psnr, plot_results, print_max_diff

xp=cp

In [ ]:
#load image
img = skimage.data.camera() #.astype(np.float32)
img = img / 255  # normalize to [0,1]

filters = [{'name':'Gabor', 'noise_level':100, 'sigma':(1000,0.1), 'theta':0}]
maxit = 5
beta = 10.

In [ ]:
# Add vertical stripes (Dirac filter)
noisy_img = stripes_addition(img, amplitude=0.2)

In [ ]:
%timeit measure_vsnr_cuda(noisy_img, filters, nite=maxit, beta=beta)
img_corr_cuda = measure_vsnr_cuda(noisy_img, filters, nite=maxit, beta=beta)

In [ ]:
if xp==cp:
    %timeit measure_vsnr_cupy(noisy_img, filters, nite=maxit, beta=beta)
    img_corr_py = measure_vsnr_cupy(noisy_img, filters, nite=maxit, beta=beta)
else:
    %timeit measure_vsnr_numpy(noisy_img, filters, nite=maxit, beta=beta)
    img_corr_py = measure_vsnr_numpy(noisy_img, filters, nite=maxit, beta=beta)

In [ ]:
# print PSNR & save plots
print_max_diff(img_corr_cuda, img_corr_py, xp) # sometimes precision might be >1e-6
print_psnr(img, noisy_img, img_corr_py, img_corr_cuda)
plot_results(img, noisy_img, img_corr_py, img_corr_cuda, xp, save_plots=True, title='camera_stripes.png')

In [ ]:
#TEST GAUSSIAN NOISE
img=skimage.data.camera()/255

# Add gaussian noise
noise =  np.random.normal(loc=0, scale=0.1, size=img.shape)
noisy_img = img + noise

filters=[{'name':'Dirac', 'noise_level':0.35}]
img_corr_cuda = measure_vsnr_cuda(noisy_img, filters, nite=20, beta=10.)

if xp==cp:
    img_corr_py = measure_vsnr_cupy(noisy_img, filters, nite=20, beta=10.)
else:
    img_corr_py = measure_vsnr_numpy(noisy_img, filters, nite=20, beta=10.)

print_max_diff(img_corr_cuda, img_corr_py, xp)
print_psnr(img, noisy_img, img_corr_py, img_corr_cuda)
plot_results(img, noisy_img, img_corr_py, img_corr_cuda, xp, save_plots=True, title='camera_gaussian.png')

In [ ]:
#TEST CURTAINS

# Generate image with noise
img_base=skimage.data.camera().astype(np.float32)
img_base=img_base/255
noisy_img = curtains_addition(img_base, amplitude=0.2, angle=50)

# Process image
filters = [{'name':'Gabor', 'noise_level':20, 'sigma':(3,40), 'theta':50}]
img_corr_cuda = measure_vsnr_cuda(noisy_img, filters, nite=maxit, beta=beta)
if xp==cp:
    img_corr_py = measure_vsnr_cupy(noisy_img, filters, nite=maxit, beta=beta)
    img_corr_py = img_corr_py.get()
else:
    img_corr_py = measure_vsnr_numpy(noisy_img, filters, nite=maxit, beta=beta)

print_max_diff(img_corr_cuda, img_corr_py, xp)
print_psnr(img_base, noisy_img, img_corr_py, img_corr_cuda)
plot_results(img_base, noisy_img, img_corr_py, img_corr_cuda, xp, save_plots=True, title='camera_curtains.png')

In [ ]:
###################DEBUG#############################################
# Generate image with noise
img=np.ones((512,512), dtype=np.float32)*128
img=img/255
noisy_img = stripes_addition(img, amplitude=0.2)


# Process image
filters = [{'name':'Gabor', 'noise_level':100, 'sigma':(1000,0.1), 'theta':0}]
img_corr_cuda = measure_vsnr_cuda(noisy_img, filters, nite=maxit, beta=beta)
if xp==cp:
    img_corr_py = measure_vsnr_cupy(noisy_img, filters, nite=maxit, beta=beta)
    img_corr_py = img_corr_py.get()
else:
    img_corr_py = measure_vsnr_numpy(noisy_img, filters, nite=maxit, beta=beta)

print_max_diff(img_corr_cuda, img_corr_py, xp)
print_psnr(img, noisy_img, img_corr_py, img_corr_cuda)
plot_results(img, noisy_img, img_corr_py, img_corr_cuda, xp)

In [ ]:
# Generate image with noise
img=np.ones((512,512), dtype=np.float32)*128
img=img/255
noisy_img = curtains_addition(img, amplitude=0.6, angle=50)


# Process image
filters = [{'name':'Gabor', 'noise_level':60, 'sigma':(3,40), 'theta':50}]
img_corr_cuda = measure_vsnr_cuda(noisy_img, filters, nite=maxit, beta=beta)
if xp==cp:
    img_corr_py = measure_vsnr_cupy(noisy_img, filters, nite=maxit, beta=beta)
    img_corr_py = img_corr_py.get()
else:
    img_corr_py = measure_vsnr_numpy(noisy_img, filters, nite=maxit, beta=beta)

print_max_diff(img_corr_cuda, img_corr_py, xp)
print_psnr(img, noisy_img, img_corr_py, img_corr_cuda)
plot_results(img, noisy_img, img_corr_py, img_corr_cuda, xp)

In [ ]:
# Cvg criteria analysis

# Generate image with noise
import matplotlib.pyplot as plt
from utils import peak_signal_noise_ratio
from src.pyvsnr import vsnr2d

img=skimage.data.camera().astype(np.float32)
img=img/255
noisy_img = stripes_addition(img, amplitude=0.2)

# Process image
filters = [{'name':'Gabor', 'noise_level':100, 'sigma':(1000,0.1), 'theta':0}]
maxit = 50
beta = 10.
psnr_cuda = []
for i in range(1,maxit):
    img_clean = vsnr2d(noisy_img, filters, nite=i, beta=beta, xp=cp)
    psnr_cuda.append(peak_signal_noise_ratio(img, img_clean))

    #Plotting each iteration
    # if i%4==0:
    #     plt.figure(figsize=(5, 5))
    #     plt.imshow(img_clean.get())
    #     plt.title(f"Denoised Image w/ nite={i}")
    #     plt.tight_layout()
    #     plt.show()


# Determine optimal number of iterations
nite_optimal = np.argmax(psnr_cuda) + 1

print('Optimal number of iterations: ', nite_optimal)
# Process image with optimal number of iterations
img_clean_optimal = measure_vsnr_cupy(noisy_img, filters, nite=nite_optimal, beta=beta)

if type(img_clean_optimal) == cp.ndarray:
    img_clean_optimal = img_clean_optimal.get()

# Plot PSNR graph
plt.figure(figsize=(15, 5))
plt.subplot(1, 3, 1)
plt.plot(psnr_cuda)
plt.xlabel('N Iteration')
plt.ylabel('PSNR')

# Plot original image
plt.subplot(1, 3, 2)
plt.imshow(noisy_img)
plt.title('Noisy Image')

# Plot denoised image
plt.subplot(1, 3, 3)
plt.imshow(img_clean_optimal)
plt.title(f"Denoised Image w/ optimal nite={nite_optimal}")

plt.tight_layout()
plt.show()

In [ ]:
# performance comparison between numpy and cupy and cuda based on the number of iterations
import time
from src.pyvsnr import vsnr2d_cuda

img=skimage.data.camera().astype(np.float32)
img=img/255
noisy_img = stripes_addition(img, amplitude=0.2)

# Process image
filters = [{'name':'Gabor', 'noise_level':100, 'sigma':(1000,0.1), 'theta':0}]
maxit = 50
beta = 10.
time_cuda = []
time_cupy = []
time_numpy = []

# CUDA
for i in range(1,maxit):
    start = time.perf_counter()
    vsnr2d_cuda(noisy_img, filters, nite=i, beta=beta, nblocks='auto')
    end = time.perf_counter()
    time_cuda.append(end-start)

# CuPy
for i in range(1,maxit):
    start = time.perf_counter()
    vsnr2d(noisy_img, filters, nite=i, beta=beta, xp=cp)
    end = time.perf_counter()
    time_cupy.append(end-start)

# NumPy
for i in range(1,maxit):
    start = time.perf_counter()
    vsnr2d(noisy_img, filters, nite=i, beta=beta, xp=np)
    end = time.perf_counter()
    time_numpy.append(end-start)

# Plot time taken on one graph
plt.figure(figsize=(15, 5))
plt.plot(time_numpy, label='NumPy')
plt.plot(time_cupy, label='CuPy')
plt.plot(time_cuda, label='CUDA')
plt.xlabel('N Iterations')
plt.ylabel('time taken (s)')
plt.legend()
plt.show()


In [ ]:
# performance comparison between numpy and cupy and cuda based on the size of the image
sizes = [256, 512, 1024, 2048]
    
time_cuda = []
time_cupy = []
time_numpy = []

for size in sizes:
    # Generate image with noise
    img=np.ones((size,size), dtype=np.float32)*128
    img=img/255
    noisy_img = stripes_addition(img, amplitude=0.2)

    # Process image
    filters = [{'name':'Gabor', 'noise_level':100, 'sigma':(1000,0.1), 'theta':0}]
    maxit = 10
    beta = 10.

    # CUDA
    start = time.perf_counter()
    vsnr2d_cuda(noisy_img, filters, nite=maxit, beta=beta, nblocks='auto')
    end = time.perf_counter()
    time_cuda.append(end-start)

    # CuPy
    start = time.perf_counter()
    vsnr2d(noisy_img, filters, nite=maxit, beta=beta, xp=cp)
    end = time.perf_counter()
    time_cupy.append(end-start)

    # NumPy
    start = time.perf_counter()
    vsnr2d(noisy_img, filters, nite=maxit, beta=beta, xp=np)
    end = time.perf_counter()
    time_numpy.append(end-start)

# Plot time taken on one graph
plt.figure(figsize=(15, 5))
plt.plot(sizes, time_numpy, label='NumPy')
plt.plot(sizes, time_cupy, label='CuPy')
plt.plot(sizes, time_cuda, label='CUDA')
plt.xlabel('Image Size (px)')
plt.ylabel('time (s)')
plt.legend()
plt.show()